In [29]:
import os
from dotenv import load_dotenv
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain_community.vectorstores.chroma import Chroma
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.documents.base import Document

In [30]:
def configure_qa_rag_chroma_chain_test(llm, embeddings, general_system_template):
    # RAG response
    #   System: Always talk in pirate speech.
    general_user_template = "Question:```{question}```"
    messages = [
        SystemMessagePromptTemplate.from_template(general_system_template),
        HumanMessagePromptTemplate.from_template(general_user_template),
    ]
    qa_prompt = ChatPromptTemplate.from_messages(messages)

    qa_chain = load_qa_with_sources_chain(
        llm,
        chain_type="stuff",
        prompt=qa_prompt,
    )

    # ChromaDB Knowledge Database response
    chromadb = Chroma(persist_directory="data_chroma", embedding_function=embeddings)

    kb_qa = RetrievalQAWithSourcesChain(
        combine_documents_chain=qa_chain,
        retriever=chromadb.as_retriever(search_kwargs={"k": 2}),
        reduce_k_below_max_tokens=False,
        max_tokens_limit=3375,
    )
    return kb_qa

# Define the system message templates
general_system_template_baseline = "{summaries}"
general_system_template_original = """Use the following pieces of context to answer the question at the end.
The context contains question-answer pairs and their links from Stackoverflow.
You should prefer information from accepted or more upvoted answers.
Make sure to rely on information from the answers and not on questions to provide accuate responses.
When you find particular answer in the context useful, make sure to cite it in the answer using the link.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----
{summaries}
----
Each answer you generate should contain a section at the end of links to 
Stackoverflow questions and answers you found useful, which are described under Source value.
You can only use links to StackOverflow questions that are present in the context and always
add links to the end of the answer in the style of citations.
Generate concise answers with references sources section of links to 
relevant StackOverflow questions only at the end of the answer."""
general_system_template_with_instructions = """Use the following pieces of context to answer the question at the end.
The context contains question-answer pairs in the following formmat:
Question: title
content
Answer: content
Score: score
Link: link
Make sure to rely on information from the answers and not on questions to provide accuate responses.
When you find particular answer in the context useful, make sure to cite it in the answer using the link.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----
{summaries}
----
Each answer you generate should contain a section at the end of links to 
Stackoverflow questions and answers you found useful, which are described under Source value.
You can only use links to StackOverflow questions that are present in the context and always
add links to the end of the answer in the style of citations.
Generate concise answers with references sources section of links to 
relevant StackOverflow questions only at the end of the answer."""

# Prepare the environment
load_dotenv()
url = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
llm=ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True)
embeddings = OpenAIEmbeddings()
chromadb = Chroma(persist_directory="data_chroma", embedding_function=embeddings)

# Configure the RAG model with ChromaDB baseline prompt
rag_chroma_chain_baseline = configure_qa_rag_chroma_chain_test(
    llm=llm, embeddings=embeddings, general_system_template=general_system_template_baseline
)

# Configure the RAG model with ChromaDB and additional instructions prompt
rag_chroma_chain_test = configure_qa_rag_chroma_chain_test(
    llm=llm, embeddings=embeddings, general_system_template=general_system_template_original
)

# Define a question
question = "Is there anyway to import the data to `neo4j` desktop"

In [31]:
dash = "-"*100

# Simulate vector search
query_vector = embeddings.embed_query(question)
retrieved_docs = chromadb.similarity_search_by_vector_with_relevance_scores(query_vector, k=2)
for doc in retrieved_docs:
    print(Document.dict(doc[0])['page_content'])
    print(Document.dict(doc[0])['metadata'])
    print(f"Relevance score: {doc[1]}")
    print(dash)

Question:Embedded database to neo4j desktop
I&#39;m currently using `DatabaseManagementServiceBuilder` to store `neo4j` data on my local host. Is there anyway to import the data to `neo4j` desktop or any other tool for visualisation? for example by creating a `.csv` file or dumping the data.

This is how my directories look like.

first  
![first][1]

second  
![second][2]

third  
![third][3]

I tried to replace the `/data` directory in a `neo4j` desktop database with my own `/data` directory but it didn&#39;t work (`neo4j` desktop kept giving an error which was no surprise).


[enter image description here][4]


  [1]: https://i.stack.imgur.com/QLm9Z.jpg
  [2]: https://i.stack.imgur.com/3EW2r.jpg
  [3]: https://i.stack.imgur.com/uU9xt.jpg

this is my code for connecting
  [4]: https://i.stack.imgur.com/GkIKu.png
Answer:
Looks like it is Neo4j Browser is that you&#39;re interested in, not Neo4j Desktop. Most straightforward way is to connect Neo4j Browser to your embedded database whi

In [32]:
# Get the answer from the baseline RAG model
print("## RAG model with ChromaDB and baseline prompt ##")
print(rag_chroma_chain_baseline({"question": question})["answer"])
print(dash)

# Get the answer from the RAG model with additional instructions
print("## RAG model with ChromaDB and custom prompt ##")
print(rag_chroma_chain_test({"question": question})["answer"])

## RAG model with ChromaDB and baseline prompt ##
Answer: Yes, you can import data to Neo4j Desktop using several methods. One common method is by using CSV files. You can import data from a CSV file using the `LOAD CSV` command in Cypher, Neo4j's query language. 

Here is a basic example of how to use `LOAD CSV`:

```
LOAD CSV FROM 'file:///path_to_your_file.csv' AS line
CREATE (:Person { name: line[0], age: toInteger(line[1])})
```

In this example, `line` is a list of values from a line in the CSV file. `line[0]` refers to the first value, `line[1]` refers to the second value, and so on. 

Please note that the file path in the `LOAD CSV` command is relative to the Neo4j import directory. 

Another method is by using Neo4j's import tool, which is a command line tool for importing large amounts of data. However, this tool is more complex and requires a specific data format.

You can also import data from other databases using various plugins and tools, such as the APOC library or the 